In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from rdkit.Chem import rdMolDescriptors


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, set_seed
from tqdm import tqdm
#import selfies as sf

from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import joblib

import os

import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
/home/elizaveta/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-16 00:03:27.130263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752613407.198865 3755620 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752613407.218968 3755620 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0

## Ключевые характеристики

In [2]:
def is_valid_smiles(smiles):
    if pd.isna(smiles):
        return False
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

In [3]:
def calculate_sa_score(mol):
    fp = rdMolDescriptors.GetMorganFingerprint(mol, radius=2)
    fps = fp.GetNonzeroElements()
    fragment_score = sum(fps.values()) / 100
    
    stereo_centers = len(Chem.FindPotentialStereo(mol))
    stereo_penalty = stereo_centers * 0.5
        
    ri = mol.GetRingInfo()
    macrocycle_penalty = 0
    for ring in ri.AtomRings():
        if len(ring) > 8:
            macrocycle_penalty += 0.5
        
    sa_score = fragment_score + stereo_penalty + macrocycle_penalty
        
    sa_score = max(1, min(10, sa_score))
        
    return round(sa_score, 2)

def count_lipinski_violations(mol):
    violations = 0
    violations += 1 if Descriptors.MolWt(mol) > 500 else 0
    violations += 1 if Descriptors.MolLogP(mol) > 5 else 0
    violations += 1 if Lipinski.NumHDonors(mol) > 5 else 0
    violations += 1 if Lipinski.NumHAcceptors(mol) > 10 else 0
    return violations

def has_toxicophore(mol, tox_filter):
    return tox_filter.HasMatch(mol)

def setup_tox_filter():
    params = FilterCatalogParams()
    params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
    return FilterCatalog(params)

In [4]:
def calculate_molecule_properties(smiles, tox_filter):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None
        
    return {
        "SMILES": smiles,
        "qed": QED.qed(mol),
        "sa_score": calculate_sa_score(mol),
        "lipinski_violations": count_lipinski_violations(mol),
        "toxicophore": has_toxicophore(mol, tox_filter)
    }

def meets_selection_criteria(props):
    return (0.5 <= props['qed'] <= 1.0 and
            2 < props['sa_score'] < 6 and
            props['lipinski_violations'] <= 1 and
            not props['toxicophore'])

def filter_molecules(smiles_list):
    tox_filter = setup_tox_filter()
    results = []
    
    for smi in tqdm(smiles_list, desc="Filtering molecules"):
        props = calculate_molecule_properties(smi, tox_filter)
        if props and meets_selection_criteria(props):
            results.append(props)
    
    return pd.DataFrame(results)

In [5]:
def calculate_bbb_permeability(smiles):
    """Расчет вероятности прохождения через ГЭБ по SMILES"""
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None
    
    # Основные дескрипторы, влияющие на прохождение через ГЭБ
    mw = Descriptors.MolWt(mol)  # Молекулярный вес
    logp = Descriptors.MolLogP(mol)  # LogP
    hbd = Descriptors.NumHDonors(mol)  # Доноры водородных связей
    hba = Descriptors.NumHAcceptors(mol)  # Акцепторы водородных связей
    psa = Descriptors.TPSA(mol)  # Полярная площадь поверхности
    
    # Эмпирические правила для ГЭБ
    # 1. Правило Липинского (адаптированное для ГЭБ)
    rule_of_five = (mw <= 500 and logp <= 5 and hbd <= 5 and hba <= 10)
    
    # 2. Дополнительные критерии
    bbb_rule = (mw <= 450 and logp > -1 and logp < 5 and 
                psa < 90 and hbd <= 3)
    
    return {
        'MW': mw,
        'LogP': logp,
        'HBD': hbd,
        'HBA': hba,
        'TPSA': psa,
        'BBB_permeable': bbb_rule,
        'Rule_of_Five': rule_of_five
    }

## Загрузка данных

Данные берутся из запусков генерации различных конфигураций MolHF

In [6]:
path = 'data//MolHF_results/'

In [7]:
def read_txt_files_to_dataframe(directory):
    df = pd.DataFrame(columns=['smiles', 'value'])
    
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.txt'):
                filepath = os.path.join(root, filename)
                
                with open(filepath, 'r') as file:
                    for line in file:
                        line = line.strip()
                        if line:
                            parts = line.split(',')
                            smiles = parts[0].strip()
                            
                            value = int(parts[1].strip()) if len(parts) > 1 else None
                            
                            df.loc[len(df)] = [smiles, value]
    
    return df

In [8]:
df = read_txt_files_to_dataframe(path)

In [9]:
df = df.drop_duplicates(subset=['smiles'])

Сгенерировано всего 7465 молекул

In [10]:
len(df)

7465

In [11]:
df['is_valid'] = df['smiles'].apply(is_valid_smiles)

Невалидных молекул нет, что является положительной стороной MolHF

In [12]:
df[~df['is_valid']]

,smiles,value,is_valid


## Отбор по ключевым характеристикам

In [13]:
def to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None
    return mol

In [14]:
df['mol'] = df['smiles'].apply(to_mol)

In [15]:
df['qed'] = df['mol'].apply(QED.qed)

In [16]:
df['sa_score'] = df['mol'].apply(calculate_sa_score)

[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerator
[00:03:50] DEPRECATION WARNING: please use MorganGenerat

In [17]:
df['lipinski_violations'] = df['mol'].apply(count_lipinski_violations)

In [18]:
tox_filter = setup_tox_filter()

In [19]:
def tox_check(mol):
    return has_toxicophore(mol, tox_filter)

In [20]:
df['toxicophore'] = df['mol'].apply(tox_check)

In [21]:
df['bbb_permeability'] = df['smiles'].apply(calculate_bbb_permeability)

In [22]:
def is_bbb_permeable(params):
    
    return (
        params['MW'] < 450 and
        -1 <= params['LogP'] <= 5 and
        params['HBD'] <= 3 and
        params['HBA'] <= 8 and
        params['TPSA'] < 90
    )

In [23]:
df['BBB_permeable'] = df['bbb_permeability'].apply(is_bbb_permeable)

In [43]:
molecules = df[(df['qed'] >= 0.5) & (df['qed'] <= 1.0) & (~df['toxicophore']) & (df['sa_score'] > 2.0) & (df['sa_score'] < 6.0) & (df['lipinski_violations'] <= 1.0) & (df['BBB_permeable'])]

In [44]:
len(molecules)

209

In [45]:
model = joblib.load('best_model.joblib')

In [46]:
def calculate_descriptors(smiles_list):
    desc_names = [d[0] for d in Descriptors._descList]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)
    desc_data = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            desc_data.append(calc.CalcDescriptors(mol))
        else:
            desc_data.append([np.nan] * len(desc_names))
    df_desc = pd.DataFrame(desc_data, columns=desc_names)
    df_desc = df_desc.rename(columns={'qed': 'qed_desc'})
    return df_desc

In [47]:
def preprocess_data(df):
    df_desc = calculate_descriptors(df['smiles'])
    df_processed = df.reset_index(drop=True).join(df_desc)
    
    df_processed = df_processed.drop(columns=['pValue', 'MOL'], errors='ignore')
    df_processed = df_processed.select_dtypes(include=[np.number])
    
    extreme = 1e6
    df_processed = df_processed.loc[:, (np.isfinite(df_processed).all(axis=0)) & 
                                    (df_processed.abs() < extreme).all(axis=0)]
    
    selector = VarianceThreshold(threshold=0.01)
    X_var = selector.fit_transform(df_processed)
    selected_cols = df_processed.columns[selector.get_support()]
    X_var = pd.DataFrame(X_var, columns=selected_cols, index=df_processed.index)
    
    corr = X_var.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
    X_uncorr = X_var.drop(columns=to_drop)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_uncorr)

    pca = PCA(n_components=n_components)
    pca_features = pca.fit_transform(X_scaled)
    
    return pca_features

In [48]:
scaler = joblib.load("preprocessors/scaler.joblib")
selector = joblib.load("preprocessors/variance_threshold.joblib")
pca = joblib.load("preprocessors/pca.joblib")
selected_cols = joblib.load("preprocessors/selected_cols.joblib")
uncorrelated_cols = joblib.load("preprocessors/uncorrelated_cols.joblib")
model = joblib.load("best_model.joblib")

In [49]:
n_components = 57

In [50]:
X_new_processed = preprocess_data(molecules)

In [51]:
predictions = model.predict(X_new_processed)

In [52]:
molecules['pValue'] = predictions

In [53]:
molecules = molecules[molecules['pValue'] >= 6]

In [54]:
len(molecules)

81

После фильтрации по базовым критериям, ещё остаётся 81 молекула, поэтому можем сузить рамки

In [37]:
molecules = molecules[molecules['pValue'] >= 6.5]

In [38]:
molecules = molecules[molecules['qed'] >= 0.65]

In [39]:
molecules = molecules[molecules['lipinski_violations'] == 0]

In [40]:
molecules = molecules.drop(columns=['value', 'mol']).sort_values('pValue', ascending=False)

In [41]:
len(molecules)

9

In [42]:
molecules.head()

,smiles,is_valid,qed,sa_score,lipinski_violations,toxicophore,bbb_permeability,BBB_permeable,pValue
5860,CCCC1C2C3C(CCN)C4CC3C42N1CCC,True,0.782264,4.01,0,False,"{'MW': 248.41399999999993, 'LogP': 2.480200000...",True,7.116340
1254,NC=CC1CCC2C(N)CN=CC(CN)C2C1,True,0.650990,3.48,0,False,"{'MW': 236.363, 'LogP': 0.47790000000000055, '...",False,6.978216
5716,CCC1C(C)C1CC(C)CN,True,0.661862,2.29,0,False,"{'MW': 155.28499999999997, 'LogP': 2.2634, 'HB...",True,6.901131
12598,CSCC(N)CCC1CCC(N)CNC=N1,True,0.660488,2.45,0,False,"{'MW': 244.40800000000002, 'LogP': 0.564500000...",True,6.892524
5741,CCCC1C2CC(C2C)C1CNC,True,0.702500,2.86,0,False,"{'MW': 181.32299999999998, 'LogP': 2.524100000...",True,6.865973


Сохраняем молекулы для дальнейшего докинга

In [157]:
molecules.to_csv('molecules_filtered_for_docking.csv', index=False)

In [158]:
molecules = molecules[molecules['qed'] >= 0.7]

In [159]:
molecules

,smiles,is_valid,qed,sa_score,lipinski_violations,toxicophore,bbb_permeability,BBB_permeable,pValue
5860,CCCC1C2C3C(CCN)C4CC3C42N1CCC,True,0.782264,4.01,0,False,"{'MW': 248.41399999999993, 'LogP': 2.480200000...",True,7.116340
5741,CCCC1C2CC(C2C)C1CNC,True,0.702500,2.86,0,False,"{'MW': 181.32299999999998, 'LogP': 2.524100000...",True,6.865973
6057,CC1CCC2(N)C(C)N(C)C(C)CC3CCC3(C1C)C2C1CN2CCN(C...,True,0.741474,5.82,0,False,"{'MW': 416.6980000000002, 'LogP': 3.8569000000...",True,6.741094


Отдельно сохраняем топ-3 молекулы по узким критериям

In [160]:
molecules.to_csv('molecules_filtered_top.csv', index=False)